Alot of this code comes from but is revised from
- nanogpt-lecture: [Andrej Karpathy, Github ng-video-lecture]( https://github.com/karpathy/ng-video-lecture?tab=readme-ov-file )

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
torch.__version__

'2.2.1'

In [2]:
import pandas as pd
import data_process as dp

dp.pdb_to_3Di_csv("../data", "./data.csv")
df = pd.read_csv("./data.csv")

In [3]:
train, val = dp.get_train_val_split(df["3Di"].tolist(), ratio=0.9)

In [28]:

batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
vocab_size = 20
max_iters = 2000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = dp.get_batch(train if split == "train" else val, batch_size=batch_size, block_size=block_size, device=device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class CausalSelfAttention(nn.Module):
    def __init__(self, n_embd, n_head, bias, dropout):
        super().__init__()
        assert n_embd % n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=bias)
        # output projection
        self.c_proj = nn.Linear(n_embd, n_embd, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class Block(nn.Module):
    def __init__(self, n_embd, n_head, bias, dropout):
        super().__init__()
        self.ln_1 = nn.LayerNorm(n_embd, bias=bias)
        self.attn = CausalSelfAttention(n_embd, n_head, bias, dropout)
        self.ln_2 = nn.LayerNorm(n_embd, bias=bias)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd, bias=bias),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd, bias=bias),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

class GPT(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, bias=True, n_layer=4, dropout=0.0):
        super().__init__()
        self.token_embedding_table = nn.Embedding(
            vocab_size, n_embd
        )  # (vocab_size, n_embd) (n_embd is just Channels C)
        self.position_embedding_table = nn.Embedding(
            block_size, n_embd
        )  # so each character knows where it is in the sequence
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, bias, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        """idx is the context and is shape (B, T)"""
        B, T = idx.shape

        # here we select out embeddings for all the tokens up to T and encode positions
        tok_embd = self.token_embedding_table(idx)  # (B, T, C)
        pos_embd = self.position_embedding_table(
            torch.arange(T, device=device)
        )  # select vectors based on context positions
        x = tok_embd + pos_embd

        # ATN 
        x = self.blocks(x)
        x = self.ln_f(x)

        # MLP at end
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            # generally speaking, this only happens during training
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(
                B * T
            )  # each index is the target of each row in logits
            loss = F.cross_entropy(logits, targets)

        # logits is shape (B, T, C) at inference time and (B*T, C) at train
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [33]:
model = GPT(
	vocab_size=20,
	n_embd = 32,
	n_head = 4,
	n_layer = 4,
	bias=False,
	dropout=0.0
)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.051796 M parameters


In [34]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(m)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = dp.get_batch(train, batch_size=batch_size, block_size=block_size, device=device)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 3.0943, val loss 3.1167
step 100: train loss 1.1743, val loss 1.3402
step 200: train loss 0.9849, val loss 1.3924
step 300: train loss 0.8649, val loss 1.5229
step 400: train loss 0.7090, val loss 1.8782
step 500: train loss 0.6177, val loss 2.1300
step 600: train loss 0.5366, val loss 2.3871
step 700: train loss 0.4752, val loss 2.5609
step 800: train loss 0.4248, val loss 2.9879
step 900: train loss 0.3855, val loss 3.0775
step 1000: train loss 0.3761, val loss 3.4011
step 1100: train loss 0.3268, val loss 3.4796
step 1200: train loss 0.3093, val loss 3.7033
step 1300: train loss 0.2838, val loss 3.8727
step 1400: train loss 0.2927, val loss 3.7541
step 1500: train loss 0.2787, val loss 4.0328
step 1600: train loss 0.2764, val loss 4.1858
step 1700: train loss 0.2431, val loss 4.2047
step 1800: train loss 0.2346, val loss 4.2002
step 1900: train loss 0.2057, val loss 4.4377
step 1999: train loss 0.2156, val loss 4.3754


In [37]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(dp.decode(m.generate(context, max_new_tokens=128)[0].tolist()))

ADDPVQLVVVLVVDHDLPDLSVLSSLLSSCVVQPQADPQLAGDLVSQLVPDRSSADSVLSNVLCVVLRPDDDPGSSNSSSSSVNSVSVSVRDNSSDNSSVSSVRDSSVSSVCVRDSVVVPPSVCVVPDD
